Jupyter notebook to extract features from the 2D microscope image
Written by Dominik Waibel & Niklas Kiermeyer

In [ ]:
#import dependencies
import os 
import numpy as np
from skimage.io import imread, imsave
from skimage.measure import label, regionprops
from skimage import measure
import trimesh
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import cv2
import math
from skimage.transform import resize
import numpy.linalg as linalg
from pyellipsoid import drawing
from skimage.filters import threshold_otsu
from skimage.feature import shape_index
from scipy.ndimage.measurements import center_of_mass
from skimage.filters import gaussian
import seaborn as sns
import csv
from skimage.morphology import convex_hull_image
from skimage.feature import greycomatrix, greycoprops
from skimage.measure import moments, inertia_tensor_eigvals
from skimage.feature import hog
import copy
from scipy.ndimage import gaussian_filter
from skimage.filters import gabor

In [ ]:
# set the path to the 2D segmentations ("mask") and the folder where the features should be saved
test_path = "./SHAPR_dataset/"
out_path = "./SHAPR_dataset/features/"

In [ ]:
# normalize and threshold the data using Otsu's method: 
#https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html
def norm_thres(data): 
    maxd = np.max(data)
    data = data / maxd
    data = np.nan_to_num(data)
    if np.max(data) > 0:
        thresh = threshold_otsu(data)
        binary = data > thresh
    else: 
        binary = data
    return binary*1.0

In [ ]:
import mahotas
from skimage.feature import hog

files = os.listdir(test_path + "/image/")
print("found", len(files), "files")

gabor_header = ["gabor"]
glcm_cor_header = ["glcm_cor"]
glcm_diss_header = ["glcm_diss"]

with open(out_path + '/mask_2dBF.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    #create and combine the header
    header1 = ['filename', 'mean', 'std']
    header = header1 + gabor_header + glcm_diss_header + glcm_cor_header
    print("len features", len(header))
    writer.writerow(header)
    for index, file in enumerate(files): 

        print(index, file)
        #read the data
        mask = norm_thres(np.squeeze(imread(test_path + "/mask/" + file)))

        image = imread(test_path + "/image/" + file)
        #threshold the image with the mask
        data_in = image*mask


        #calculate the glcm features
        glcm = greycomatrix(data_in.astype("uint8"), distances=[5], angles=[0], levels=256,
                            symmetric=True, normed=True)
        glcm_diss = [(greycoprops(glcm, 'dissimilarity')[0, 0])]
        glcm_cor = [(greycoprops(glcm, 'correlation')[0, 0])]
        #calculate the gabor features
        gabor_feat = [np.sum(gabor(data_in,1)[0])]
        #combine the features and write them to the .csv file
        feat = [file, np.mean(data_in), np.std(data_in)]
        print("len features", len(feat), len(header1))
        feat = feat + gabor_feat  + glcm_diss + glcm_cor
        writer.writerow(feat)     